# Genome-scale metabolic models

## Preparation

In [ ]:
from cobra.io import read_sbml_model
model = read_sbml_model('data/iJO1366.xml.gz')

## Model content

### Metabolites

The model contains a list of metabolites. Here are the first ten.

In [ ]:
model.metabolites[0:10]

There are 1805 metabolites in the model.

In [ ]:
len(model.metabolites)

One can access a specific metabolite using dot notation.

In [ ]:
model.metabolites.g3p_c

<div class="alert alert-warning">

**Warning:** One cannot use dot notation to access metabolites, reactions, or genes if their identifiers do not resemble proper Python variable names.

</div>

<div class="alert alert-success">

**Solution:** Use the method `get_by_id` instead!

</div>

In [ ]:
model.metabolites.get_by_id('10fthf_c')

Metabolites are associated with compartments in the cell. Glyceraldehyde 3-phosphate (`g3p_c`) is associated with the `c` (Cytosol) compartment.

In [ ]:
model.metabolites.g3p_c.compartment

The _E. coli_ model has three compartments.

In [ ]:
model.compartments

Some metabolites (like Glucose for example) can be associated with multiple compartments.

In [ ]:
model.metabolites.glc__D_c.compartment

In [ ]:
model.metabolites.glc__D_p.compartment

The full name of the metabolite is available via the `.name` attribute. 

In [ ]:
model.metabolites.glc__D_c.name

One can look up the molecular formula of glucose.

In [ ]:
model.metabolites.g3p_c.formula

The `.elements` attribute returns a dictionary representation of the formula.

In [ ]:
model.metabolites.g3p_c.elements

Furthermore, one can look up the molecular weight of a metabolite.

In [ ]:
model.metabolites.g3p_c.formula_weight

One can gather additional information (like references to external datbases) about the metabolite through the annotation attribute.

In [ ]:
model.metabolites.g3p_c.annotation

One can use these annotations to look up the compound on [KEGG](http://www.genome.jp/dbget-bin/www_bget?cpd:C00118) for example.

Metabolites are not isolated things. They participate in reactions as substrates and products.

In [ ]:
model.metabolites.g3p_c.reactions

### Reactions

The model contains a list of reactions. Here are the first 10 of them.

In [ ]:
model.reactions[0:10]

There are 2583 reactions in the model.

In [ ]:
len(model.reactions)

Let's take a closer look at the reactions associated with Glyceraldehyde 3-phosphate (`g3p`).

In [ ]:
for reaction in model.metabolites.g3p_c.reactions:
    print(reaction.id, reaction, reaction.name)

The second reaction in this list is Glyceraldehyde-3-phosphate dehydrogenase (GAPD).

In [ ]:
model.reactions.GAPD.name

## Objective

In [ ]:
print(model.objective)

## ATP maintenance requirement

In [ ]:
model.reactions.ATPM

## The math (scary!)

In [ ]:
print(model.solver.__str__()[0:1550])

## Gene-Protein-Reaction associations

Glyceraldehyde-3-phosphate dehydrogenase is associated with a single gene. 

In [ ]:
model.reactions.GAPD.gene_reaction_rule

Phosphofructokinase (PFK) on the other hand seems to be associated with to isozymes.

In [ ]:
model.reactions.PFK.gene_reaction_rule

One can display the gene names (typical 4 letter gene codes) instead of the identifiers (Blattner numbers in this case).

In [ ]:
model.reactions.PFK.gene_name_reaction_rule

Here a very complicated gene to reaction mapping (ATP synthase).

In [ ]:
model.reactions.ATPS4rpp.gene_name_reaction_rule

## Finding things

One can use `.query('search term', 'attribute_to_search_in')` to search in model metabolites, reactions, and genes. For example, one can search metabolites that contain the term _glucose_ in their name.

In [ ]:
for metabolite in model.metabolites.query('glucose', 'name'):
    print(metabolite.name)

## The stoichiometric matrix S

In [ ]:
from cobra.util import create_stoichiometric_matrix
s = create_stoichiometric_matrix(model)
s

This is how the the stoichiometry matrix S looks like when visualized as a matrix plot.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.spy(s, precision=0.01, markersize=.1)

## Simulating models

In [ ]:
model.optimize()

Sometimes a solution cannot be found. For example, setting the lower bound of the objective function to a very high value that the model cannot achieve will trigger a warning when trying to optimize the model. Parameters reported from an infeasible model are not meaningful to interpret (except in rare occasions when you may want to figure out why a model is infeasible).

In [ ]:
infeasible_model = model.copy()
infeasible_model.reactions.BIOMASS_Ec_iJO1366_core_53p95M.lower_bound = 100000
sol = infeasible_model.optimize()